<a href="https://colab.research.google.com/github/Yewon9/STT_JEJU/blob/main/Wav2Vec2_Fine_Tuning_Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORT

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
!pip install transformers datasets torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.6 MB/s eta 0:00:00


In [21]:
import pandas as pd
import torch
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from datasets import load_dataset, Dataset, DatasetDict
from transformers import Trainer, TrainingArguments

# DATA sample

In [13]:
df = pd.read_csv('df_eda.csv')
df

,Path,Text,Standard Text,Audio_Length,Text_Length
0,/content/drive/MyDrive/JEJU/data/wav/TL_02/say...,여름에 보양식은 그 동네에서 보면 그 자리들 잡아가지고 된장에 풀어가지고 생된장에 ...,여름에 보양식은 그 동네에서 보면 그 자리돔들 잡아서 된장에 풀어서 생된장에 풀어서...,37.309,219
1,/content/drive/MyDrive/JEJU/data/wav/TL_02/say...,태풍은 나 중학교 땐가 정확하게 기억은 잘 안 나는디 옛날에 해가지고는 엄청나게 사...,태풍은 나 중학교 때인가 정확하게 기억은 잘 안 나는데 옛날에 해가지고는 엄청나게 ...,36.210,211
2,/content/drive/MyDrive/JEJU/data/wav/TL_02/say...,그추룩헹은에 이제 제사 지낸 지가 한 이제 오 년 정도 됐신디예 어떵 하다 보니까 ...,그렇게 해서 이제 제사 지낸 지가 한 이제 오 년 정도 됐는데요 어떻게 하다 보니까...,55.159,417
3,/content/drive/MyDrive/JEJU/data/wav/TL_02/say...,어 여기는 지금 농작물은 농사를 뭐 밭농사랑 뭐 과수원 같은 거 많이 하니까 뭐 이...,어 여기는 지금 농작물은 농사를 뭐 밭농사랑 뭐 과수원 같은 거 많이 하니까 뭐 이...,40.959,242
4,/content/drive/MyDrive/JEJU/data/wav/TL_02/say...,우리는 형제 자매가 많아 가지고 옛날이라부난 옛날에 어른들은 게 아이들 형제들 네 ...,우리는 형제 자매가 많아 가지고 옛날이다보니까 옛날에 어른들은 게 아이들 형제들 네...,37.250,224
...,...,...,...,...,...
11989,/content/drive/MyDrive/JEJU/data/wav/VL_01/st_...,이제 눈 어두웡 바농귀 끼지 못행 바농질 못 하켜,이제 눈 어두워서 바늘귀 꿰지 못해서 바느질 못 하겠어,5.040,27
11990,/content/drive/MyDrive/JEJU/data/wav/VL_01/st_...,밧디서 나록을 비민 무껑 그 나록뭇을 쉐에 시껑 집이 왓저,밭에서 벼를 베면 묶어서 그 볏단을 소에 싣고 집에 왔지,7.680,32
11991,/content/drive/MyDrive/JEJU/data/wav/VL_01/st_...,일 끗나거덜랑 어드레 가지 말앙 고짝 집더레 옵서예,일 끝나거든 어디 가지 말고 곧장 집으로 오세요,5.700,28
11992,/content/drive/MyDrive/JEJU/data/wav/VL_01/st_...,나가 그자 온 것이 아니고 이 집이 큰똘을 중진허젠 허로 왓수다,내가 그냥 온 것이 아니고 이 집이 큰딸을 중신하려고 하러 왔습니다,4.940,35


In [14]:
over_100_seconds = df[df['Audio_Length'] > 60]
print(f"100초 이상인 데이터 개수: {len(over_100_seconds)}")

100초 이상인 데이터 개수: 1378


In [15]:
df_filtered = df[df['Audio_Length'] <= 60]
df_filtered

,Path,Text,Standard Text,Audio_Length,Text_Length
0,/content/drive/MyDrive/JEJU/data/wav/TL_02/say...,여름에 보양식은 그 동네에서 보면 그 자리들 잡아가지고 된장에 풀어가지고 생된장에 ...,여름에 보양식은 그 동네에서 보면 그 자리돔들 잡아서 된장에 풀어서 생된장에 풀어서...,37.309,219
1,/content/drive/MyDrive/JEJU/data/wav/TL_02/say...,태풍은 나 중학교 땐가 정확하게 기억은 잘 안 나는디 옛날에 해가지고는 엄청나게 사...,태풍은 나 중학교 때인가 정확하게 기억은 잘 안 나는데 옛날에 해가지고는 엄청나게 ...,36.210,211
2,/content/drive/MyDrive/JEJU/data/wav/TL_02/say...,그추룩헹은에 이제 제사 지낸 지가 한 이제 오 년 정도 됐신디예 어떵 하다 보니까 ...,그렇게 해서 이제 제사 지낸 지가 한 이제 오 년 정도 됐는데요 어떻게 하다 보니까...,55.159,417
3,/content/drive/MyDrive/JEJU/data/wav/TL_02/say...,어 여기는 지금 농작물은 농사를 뭐 밭농사랑 뭐 과수원 같은 거 많이 하니까 뭐 이...,어 여기는 지금 농작물은 농사를 뭐 밭농사랑 뭐 과수원 같은 거 많이 하니까 뭐 이...,40.959,242
4,/content/drive/MyDrive/JEJU/data/wav/TL_02/say...,우리는 형제 자매가 많아 가지고 옛날이라부난 옛날에 어른들은 게 아이들 형제들 네 ...,우리는 형제 자매가 많아 가지고 옛날이다보니까 옛날에 어른들은 게 아이들 형제들 네...,37.250,224
...,...,...,...,...,...
11989,/content/drive/MyDrive/JEJU/data/wav/VL_01/st_...,이제 눈 어두웡 바농귀 끼지 못행 바농질 못 하켜,이제 눈 어두워서 바늘귀 꿰지 못해서 바느질 못 하겠어,5.040,27
11990,/content/drive/MyDrive/JEJU/data/wav/VL_01/st_...,밧디서 나록을 비민 무껑 그 나록뭇을 쉐에 시껑 집이 왓저,밭에서 벼를 베면 묶어서 그 볏단을 소에 싣고 집에 왔지,7.680,32
11991,/content/drive/MyDrive/JEJU/data/wav/VL_01/st_...,일 끗나거덜랑 어드레 가지 말앙 고짝 집더레 옵서예,일 끝나거든 어디 가지 말고 곧장 집으로 오세요,5.700,28
11992,/content/drive/MyDrive/JEJU/data/wav/VL_01/st_...,나가 그자 온 것이 아니고 이 집이 큰똘을 중진허젠 허로 왓수다,내가 그냥 온 것이 아니고 이 집이 큰딸을 중신하려고 하러 왔습니다,4.940,35


In [16]:
df_sampled = df_filtered.sample(n = 5000, random_state = 42)
df_sampled

,Path,Text,Standard Text,Audio_Length,Text_Length
10979,/content/drive/MyDrive/JEJU/data/wav/VL_01/st_...,경 들럭퀴지 말앙 초근초근 말을 들엉 허라,그렇게 날뛰지 말고 차근차근 말을 듣고 해라,3.88,23
7712,/content/drive/MyDrive/JEJU/data/wav/VL_01/st_...,옛날은 약이 엇이난 머리 아프덴 약 먹곡 헤져샤게,옛날은 약이 없으니 머리 아프다고 약 먹고 할 수 있었니,5.95,27
4681,/content/drive/MyDrive/JEJU/data/wav/TL_02/say...,뭐 온돌방이나 보일라 방이나 겨울에 추위 피해젠헹으네 허는 건디 어릴 때 온돌은 어...,뭐 온돌방이나 보일러 방이나 겨울에 추위 피하자고 해서 하는 건데 어릴 때 온돌은 ...,22.62,115
2945,/content/drive/MyDrive/JEJU/data/wav/TL_02/say...,아이고 사람 살리젠 허민 일일구에 빨리 신고해사주게 어떵허여게,아이고 사람 살리려고 하면 일일구에 빨리 신고해야지 어떻게하겠어,5.15,34
10829,/content/drive/MyDrive/JEJU/data/wav/VL_01/st_...,저디 콕박세기에 물 호꼼 거령 오라,저기 박으로 만든 바가지에 물 조금 떠 와라,4.11,19
...,...,...,...,...,...
4024,/content/drive/MyDrive/JEJU/data/wav/TL_02/say...,아 이번에 오월달에 친구들 여섯 명이 저 사회 친구들 여섯 명이 꽃 구경 가신디 오...,아 이번에 오월달에 친구들 여섯 명이 저 사회 친구들 여섯 명이 꽃 구경 가는데 오...,44.32,289
1350,/content/drive/MyDrive/JEJU/data/wav/TL_02/say...,우리도 수출 하젠하민 어쩔 수 없이 수입도 해야 되지 않을 건가 헌디 난 중국 애들...,우리도 수출 하려고하면 어쩔 수 없이 수입도 해야 되지 않을 까 그런데 난 중국 애...,15.28,67
10986,/content/drive/MyDrive/JEJU/data/wav/VL_01/st_...,자이 폴에 독솔 막 난 거 보난 언 생이우다,쟤 팔에 닭살 많이 난 거 보니까 추운 모양입니다,4.74,24
10636,/content/drive/MyDrive/JEJU/data/wav/VL_01/st_...,요새 아이덜 너미 호강 햄져 두린 때 고생도 허고 해사 큰 사름 되주게,요새 아이들 너무 호강 하고 있어 어릴 때 고생도 하고 해야 큰 사람 된다,6.55,39


In [17]:
df_remaining = df_filtered.drop(df_sampled.index)
df_remaining.shape

(5616, 5)

In [18]:
df_sampled.to_csv('sampled_5000_data.csv', index = False, encoding = "utf-8-sig")

In [19]:
df_remaining.to_csv('remaining_5000_data.csv', index = False, encoding = "utf-8-sig")

# 토크나이저와 모델 불러오기

In [22]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h", ctc_loss_reduction="mean", pad_token_id=processor.tokenizer.pad_token_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

# DATA

In [23]:
data = pd.read_csv('sampled_5000_data.csv')
data

,Path,Text,Standard Text,Audio_Length,Text_Length
0,/content/drive/MyDrive/JEJU/data/wav/VL_01/st_...,경 들럭퀴지 말앙 초근초근 말을 들엉 허라,그렇게 날뛰지 말고 차근차근 말을 듣고 해라,3.88,23
1,/content/drive/MyDrive/JEJU/data/wav/VL_01/st_...,옛날은 약이 엇이난 머리 아프덴 약 먹곡 헤져샤게,옛날은 약이 없으니 머리 아프다고 약 먹고 할 수 있었니,5.95,27
2,/content/drive/MyDrive/JEJU/data/wav/TL_02/say...,뭐 온돌방이나 보일라 방이나 겨울에 추위 피해젠헹으네 허는 건디 어릴 때 온돌은 어...,뭐 온돌방이나 보일러 방이나 겨울에 추위 피하자고 해서 하는 건데 어릴 때 온돌은 ...,22.62,115
3,/content/drive/MyDrive/JEJU/data/wav/TL_02/say...,아이고 사람 살리젠 허민 일일구에 빨리 신고해사주게 어떵허여게,아이고 사람 살리려고 하면 일일구에 빨리 신고해야지 어떻게하겠어,5.15,34
4,/content/drive/MyDrive/JEJU/data/wav/VL_01/st_...,저디 콕박세기에 물 호꼼 거령 오라,저기 박으로 만든 바가지에 물 조금 떠 와라,4.11,19
...,...,...,...,...,...
4995,/content/drive/MyDrive/JEJU/data/wav/TL_02/say...,아 이번에 오월달에 친구들 여섯 명이 저 사회 친구들 여섯 명이 꽃 구경 가신디 오...,아 이번에 오월달에 친구들 여섯 명이 저 사회 친구들 여섯 명이 꽃 구경 가는데 오...,44.32,289
4996,/content/drive/MyDrive/JEJU/data/wav/TL_02/say...,우리도 수출 하젠하민 어쩔 수 없이 수입도 해야 되지 않을 건가 헌디 난 중국 애들...,우리도 수출 하려고하면 어쩔 수 없이 수입도 해야 되지 않을 까 그런데 난 중국 애...,15.28,67
4997,/content/drive/MyDrive/JEJU/data/wav/VL_01/st_...,자이 폴에 독솔 막 난 거 보난 언 생이우다,쟤 팔에 닭살 많이 난 거 보니까 추운 모양입니다,4.74,24
4998,/content/drive/MyDrive/JEJU/data/wav/VL_01/st_...,요새 아이덜 너미 호강 햄져 두린 때 고생도 허고 해사 큰 사름 되주게,요새 아이들 너무 호강 하고 있어 어릴 때 고생도 하고 해야 큰 사람 된다,6.55,39


In [24]:
dataset = Dataset.from_pandas(data)

# 음성 파일 전처리 함수

In [25]:
def speech_file_to_array_fn(batch):
    # 음성 파일을 로드하고 16kHz로 리샘플링
    speech_array, sampling_rate = torchaudio.load(batch["Path"])
    resampler = torchaudio.transforms.Resample(orig_freq=sampling_rate, new_freq=16000)
    batch["speech"] = resampler(speech_array).squeeze().numpy()
    return batch

In [ ]:
dataset = dataset.map(speech_file_to_array_fn)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

# 토크나이저로 텍스트 변환

In [ ]:
def tokenize_text(batch):
    # 음성 데이터를 모델에 맞게 토큰화
    batch["input_values"] = processor(batch["speech"], return_tensors="pt", padding="longest").input_values
    # 텍스트 데이터를 토큰화
    with processor.as_target_processor():
        batch["labels"] = processor(batch["Cleaned_Text"], return_tensors="pt", padding="longest").input_ids
    return batch

In [ ]:
dataset = dataset.map(tokenize_text, remove_columns=["speech", "Cleaned_Text"])

# 데이터셋 분리

In [ ]:
train_testvalid = dataset.train_test_split(test_size=0.2)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)

In [ ]:
dataset_dict = DatasetDict({
    'train': train_testvalid['train'],
    'validation': test_valid['test'],
    'test': test_valid['train']
})

# Fine Tuning

In [ ]:
training_args = TrainingArguments(
    output_dir="./wav2vec2-finetuning",
    evaluation_strategy="epoch",
    learning_rate=1e-4,  # Fine-tuning에 적합한 학습률 설정
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,  # 3번의 epoch
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_total_limit=2,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_dict["train"],
    eval_dataset=dataset_dict["validation"],
    tokenizer=processor.feature_extractor,
)

# 모델 학습

In [ ]:
trainer.train()

# 모델 평가

In [ ]:
results = trainer.evaluate(dataset_dict["test"])
print(f"Test Results: {results}")

# 모델 저장

In [ ]:
model.save_pretrained('./wav2vec2-finetuned-model')
processor.save_pretrained('./wav2vec2-finetuned-processor'